In [1]:
from tensorflow.contrib.keras.python.keras.models import Sequential, Model
from tensorflow.contrib.keras.python.keras.layers.recurrent import SimpleRNN, LSTM, GRU
from tensorflow.contrib.keras.python.keras.layers.core import Dense, Dropout, Flatten
from tensorflow.contrib.keras.python.keras import layers
from tensorflow.contrib.keras.python.keras.layers import Input, Reshape, ZeroPadding2D, Conv2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D
from tensorflow.contrib.keras.python.keras.layers.wrappers import TimeDistributed

from tensorflow.contrib.keras.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.contrib.keras.python.keras.optimizers import RMSprop, Adadelta, Adam

from tensorflow.contrib.keras.python.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.contrib.keras.python.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.contrib.keras.python.keras.models import load_model
from tensorflow.contrib.keras.python.keras.preprocessing import image

from tensorflow.contrib.keras.python.keras.layers.core import K
K.set_learning_phase(0)

import numpy as np
import os
import glob

from IPython.display import clear_output

# Confusion matrix
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

import time
import pickle

In [2]:
# network
TRAIN = 1
n_classes = 7
# n_classes = 6
classes=np.array(('angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'))
# classes=np.array(('angry', 'disgust', 'fear', 'neutral', 'sad', 'surprise'))

# data
REBUILD_DATA = 0
size = 224
video_length = 48
features_length = 2048

# training
PATH_TRAIN = 'E:/EmotiW2017/lstm/aligned/Train_AFEW_original'
# PATH_TRAIN = 'E:/EmotiW2017/lstm/aligned/Train_AFEW_original'
# PATH_TRAIN = 'E:/EmotiW2017/lstm/aligned/Train_AFEW_N'
NPY_X_TRAIN_DATA = 'training_data_x_resnet50_aligned.npy'
NPY_Y_TRAIN_DATA = 'training_data_y_resnet50_aligned.npy'
# NPY_X_TRAIN_DATA = 'training_data_x_resnet50_aligned_aug.npy'
# NPY_Y_TRAIN_DATA = 'training_data_y_resnet50_aligned_aug.npy'
# NPY_X_TRAIN_DATA = 'training_data_x_inceptionv3_aligned.npy'
# NPY_Y_TRAIN_DATA = 'training_data_y_inceptionv3_aligned.npy'

# validation
# PATH_VAL = 'E:/EmotiW2017/lstm/aligned/Val_AFEW_original'
PATH_VAL = 'E:/EmotiW2017/lstm/aligned/Val_AFEW_original'
NPY_X_VAL_DATA = 'val_data_x_resnet50_aligned.npy'
NPY_Y_VAL_DATA = 'val_data_y_resnet50_aligned.npy'
# NPY_X_VAL_DATA = 'val_data_x_inceptionv3_aligned.npy'
# NPY_Y_VAL_DATA = 'val_data_y_inceptionv3_aligned.npy'

# model
WEIGHT_FILE = 'resnet50_lstm_aligned_3.hdf5'
MODEL_FILE = 'resnet50_lstm_aligned_3.h5'
TRAINING_HIST = 'resnet50_lstm_aligned_3.pickle'
# WEIGHT_FILE = 'inceptionv3_lstm_aligned.hdf5'
# MODEL_FILE = 'inceptionv3_lstm_aligned.h5'
# TRAINING_HIST = 'inceptionv3_lstm_aligned.pickle'

# Feature Extraction with ResNet50

In [ ]:
model = load_model('ResNet50_ImageNet.hdf5')
# model = load_model('InceptionV3_ImageNet.hdf5')
# model = InceptionV3(weights='imagenet', include_top=False)
# model.save('InceptionV3_ImageNet.hdf5')

In [ ]:
print(model.summary())

In [ ]:
def extract_feature(img_path, model, size):
    img = image.load_img(img_path, target_size=(size, size))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    features = model.predict(x)
    
    return features[0][0][0]

features = extract_feature('duong.jpg', model, size)
features.shape

# Emotion to Integer Convertor

In [ ]:
def emo2int(emo):
    if n_classes == 7:
        if emo == 'Angry':
            return 0
        elif emo == 'Disgust':
            return 1
        elif emo == 'Fear':
            return 2
        elif emo == 'Happy':
            return 3
        elif emo == 'Neutral':
            return 4
        elif emo == 'Sad':
            return 5
        else:
            return 6
    elif n_classes == 6:
        if emo == 'Angry':
            return 0
        elif emo == 'Disgust':
            return 1
        elif emo == 'Fear':
            return 2
        elif emo == 'Neutral':
            return 3
        elif emo == 'Sad':
            return 4
        else:
            return 5
    else:
        if emo == 'angry_happy_neutral':
            return 0
        else:
            return 1

# Confusion matrix

In [ ]:
def plot_confusion_matrix(y_test, y_pred, classes,
                          normalize=False,
                          title='Unnormalized confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    cm = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)
    
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = np.round(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], 2)

    thresh = cm.min() + (cm.max() - cm.min()) / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

# Training Data Feature Extraction

In [ ]:
# if REBUILD_DATA == 0 and os.path.isfile(NPY_X_TRAIN_DATA) and os.path.isfile(NPY_Y_TRAIN_DATA):
#     print('>>> loading training data...', end='')
#     X_train = np.load(NPY_X_TRAIN_DATA)
#     y_train = np.load(NPY_Y_TRAIN_DATA)
#     print('done.')
# else:
#     # Count # of videos
#     n_vids_train = 0

#     for emo in os.listdir(PATH_TRAIN):
#         full_emo = PATH_TRAIN + '/' + emo
#         if os.path.isdir(full_emo):
#             for vid in os.listdir(full_emo):
#                 full_vid = full_emo + '/' + vid

#                 n_vids_train += 1
    
#     # Read images and extract features
#     vid_indx = 0
    
#     X_train = np.zeros((n_vids_train, video_length, features_length))
#     y_train = []

#     for emo in os.listdir(PATH_TRAIN):
#         full_emo = PATH_TRAIN + '/' + emo
#         if os.path.isdir(full_emo):
#             for vid in os.listdir(full_emo):
#                 full_vid = full_emo + '/' + vid

#                 # Extract features
#                 fra_indx = 0
#                 for fra in os.listdir(full_vid):
#                     if fra_indx < video_length:
#                         full_fra = full_vid + '/' + fra
#                         if os.path.isfile(full_fra):
#                             # Clear screen
#                             clear_output(wait=True)
#                             print('> extracting features for video #%d/%d at frame #%d/%d' % (vid_indx + 1, n_vids_train, fra_indx + 1, video_length))
                            
#                             features = extract_feature(full_fra, model, size)
                            
#                             # Add extracted features to dataset
#                             X_train[vid_indx, fra_indx, :] = features

#                             fra_indx += 1
#                     else:
#                         break

#                 # Get the label for current video
#                 y_train.append(emo2int(emo))

#                 vid_indx += 1

#     y_train = np.array(y_train)
    
#     clear_output(wait=True)
#     print('>>> processed %d videos' % (n_vids_train))

#     np.save(NPY_X_TRAIN_DATA, X_train)
#     np.save(NPY_Y_TRAIN_DATA, y_train)

print('>>> loading training data...', end='')
X_train = np.load(NPY_X_TRAIN_DATA)
y_train = np.load(NPY_Y_TRAIN_DATA)
print('done.')
    
print('\tshapes: {}, {}'.format(X_train.shape, y_train.shape))
print('\ttypes:  {}, {}'.format(X_train.dtype, y_train.dtype))
print('\tmemory: {} MB, {} MB'.format(X_train.nbytes / 1048576, y_train.nbytes / 1048576))

# Validation Data Feature Extraction

In [ ]:
if REBUILD_DATA == 0 and os.path.isfile(NPY_X_VAL_DATA) and os.path.isfile(NPY_Y_VAL_DATA):
    print('>>> loading validation data...', end='')
    X_val = np.load(NPY_X_VAL_DATA)
    y_val = np.load(NPY_Y_VAL_DATA)
    print('done.')
else:
    # Count # of videos
    n_vids_val = 0

    for emo in os.listdir(PATH_VAL):
        full_emo = PATH_VAL + '/' + emo
        if os.path.isdir(full_emo):
            for vid in os.listdir(full_emo):
                full_vid = full_emo + '/' + vid

                n_vids_val += 1
    
    # Read images and extract features
    vid_indx = 0
    
    X_val = np.zeros((n_vids_val, video_length, features_length))
    y_val = []

    for emo in os.listdir(PATH_VAL):
        full_emo = PATH_VAL + '/' + emo
        if os.path.isdir(full_emo):
            for vid in os.listdir(full_emo):
                full_vid = full_emo + '/' + vid

                # Extract features
                fra_indx = 0
                for fra in os.listdir(full_vid):
                    if fra_indx < video_length:
                        full_fra = full_vid + '/' + fra
                        if os.path.isfile(full_fra):
                            # Clear screen
                            clear_output(wait=True)
                            print('> extracting features for video #%d/%d at frame #%d/%d' % (vid_indx + 1, n_vids_val, fra_indx + 1, video_length))
                            
                            features = extract_feature(full_fra, model, size)
                            
                            # Add extracted features to dataset
                            X_val[vid_indx, fra_indx, :] = features

                            fra_indx += 1
                    else:
                        break

                # Get the label for current video
                y_val.append(emo2int(emo))

                vid_indx += 1

    y_val = np.array(y_val)
    
    clear_output(wait=True)
    print('>>> processed %d videos' % (n_vids_val))

    np.save(NPY_X_VAL_DATA, X_val)
    np.save(NPY_Y_VAL_DATA, y_val)
    
print('\tshapes: {}, {}'.format(X_val.shape, y_val.shape))
print('\ttypes:  {}, {}'.format(X_val.dtype, y_val.dtype))
print('\tmemory: {} MB, {} MB'.format(X_val.nbytes / 1048576, y_val.nbytes / 1048576))

In [ ]:
y_val

# LSTM

In [ ]:
lstm = Sequential()
lstm.add(LSTM(128, input_shape=(video_length, features_length),
               dropout=0.5, return_sequences=False))
# lstm.add(LSTM(2048, input_shape=(video_length, features_length),
#                dropout=0.5, return_sequences=False))
# lstm.add(Flatten())
lstm.add(Dense(256, activation='relu'))
lstm.add(Dropout(0.5))
lstm.add(Dense(256, activation='relu'))
lstm.add(Dropout(0.5))
lstm.add(Dense(n_classes, activation='softmax'))

In [ ]:
print(basemodel.summary())

# ResNet + LSTM

In [4]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.

    Arguments:
      input_tensor: input tensor
      kernel_size: defualt 3, the kernel size of middle conv layer at main path
      filters: list of integers, the filterss of 3 conv layer at main path
      stage: integer, current stage label, used for generating layer names
      block: 'a','b'..., current block label, used for generating layer names

    Returns:
      Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = TimeDistributed(Conv2D(filters1, (1, 1), name=conv_name_base + '2a'))(input_tensor)
    x = TimeDistributed(BatchNormalization(axis=bn_axis, name=bn_name_base + '2a'))(x)
    x = TimeDistributed(Activation('relu'))(x)

    x = TimeDistributed(Conv2D(
        filters2, kernel_size, padding='same', name=conv_name_base + '2b'))(x)
    x = TimeDistributed(BatchNormalization(axis=bn_axis, name=bn_name_base + '2b'))(x)
    x = TimeDistributed(Activation('relu'))(x)

    x = TimeDistributed(Conv2D(filters3, (1, 1), name=conv_name_base + '2c'))(x)
    x = TimeDistributed(BatchNormalization(axis=bn_axis, name=bn_name_base + '2c'))(x)

    x = layers.add([x, input_tensor])
    x = TimeDistributed(Activation('relu'))(x)
    
    return x

In [5]:
def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """conv_block is the block that has a conv layer at shortcut.

    Arguments:
      input_tensor: input tensor
      kernel_size: defualt 3, the kernel size of middle conv layer at main path
      filters: list of integers, the filterss of 3 conv layer at main path
      stage: integer, current stage label, used for generating layer names
      block: 'a','b'..., current block label, used for generating layer names
      strides: Tuple of integers.

    Returns:
      Output tensor for the block.

    Note that from stage 3, the first conv layer at main path is with
    strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = TimeDistributed(Conv2D(
        filters1, (1, 1), strides=strides,
        name=conv_name_base + '2a'))(input_tensor)
    x = TimeDistributed(BatchNormalization(axis=bn_axis, name=bn_name_base + '2a'))(x)
    x = TimeDistributed(Activation('relu'))(x)

    x = TimeDistributed(Conv2D(
        filters2, kernel_size, padding='same', name=conv_name_base + '2b'))(x)
    x = TimeDistributed(BatchNormalization(axis=bn_axis, name=bn_name_base + '2b'))(x)
    x = TimeDistributed(Activation('relu'))(x)

    x = TimeDistributed(Conv2D(filters3, (1, 1), name=conv_name_base + '2c'))(x)
    x = TimeDistributed(BatchNormalization(axis=bn_axis, name=bn_name_base + '2c'))(x)

    shortcut = TimeDistributed(Conv2D(
        filters3, (1, 1), strides=strides,
        name=conv_name_base + '1'))(input_tensor)
    shortcut = TimeDistributed(BatchNormalization(axis=bn_axis, name=bn_name_base + '1'))(shortcut)

    x = layers.add([x, shortcut])
    x = TimeDistributed(Activation('relu'))(x)
    
    return x

In [21]:
bn_axis = 3
img_input = Input(shape=(video_length, 224, 224, 3))

x = TimeDistributed(ZeroPadding2D((3, 3)))(img_input)
x = TimeDistributed(Conv2D(64, (7, 7), strides=(2, 2), name='conv1'))(x)
x = TimeDistributed(BatchNormalization(axis=bn_axis, name='bn_conv1'))(x)
x = TimeDistributed(Activation('relu'))(x)
x = TimeDistributed(MaxPooling2D((3, 3), strides=(2, 2)))(x)

x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

x = TimeDistributed(AveragePooling2D((7, 7), name='avg_pool'))(x)

x = TimeDistributed(Flatten())(x)

# Create base model (ResNet)
base_model = Model(img_input, x, name='resnet50_lstm')
print(base_model.summary())

# Load ImageNet weights
weights_path = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model.load_weights(weights_path)

(?, 48, 2048)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_11 (InputLayer)            (None, 48, 224, 224,  0                                            
____________________________________________________________________________________________________
time_distributed_1584 (TimeDistr (None, 48, 230, 230,  0                                            
____________________________________________________________________________________________________
time_distributed_1585 (TimeDistr (None, 48, 112, 112,  9472                                         
____________________________________________________________________________________________________
time_distributed_1586 (TimeDistr (None, 48, 112, 112,  256                                          
_____________________________________________________________________________

In [13]:
x = base_model.output
print(x.shape)

x = LSTM(128, input_shape=(video_length, 1, 1, features_length), dropout=0.5, return_sequences=False)(x)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(n_classes, activation='softmax')(x)

# # Create model.
# model = Model(img_input, x, name='resnet50_lstm')

(?, 48, 1, 1, 2048)


ValueError: Input 0 is incompatible with layer lstm_4: expected ndim=3, found ndim=5

In [7]:
print(base_model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 48, 224, 224,  0                                            
____________________________________________________________________________________________________
time_distributed_1 (TimeDistribu (None, 48, 230, 230,  0                                            
____________________________________________________________________________________________________
time_distributed_2 (TimeDistribu (None, 48, 112, 112,  9472                                         
____________________________________________________________________________________________________
time_distributed_3 (TimeDistribu (None, 48, 112, 112,  256                                          
___________________________________________________________________________________________

In [ ]:
batch_size = 172
n_epochs = 10000

learning_rate = 0.0001
early_stop_after = 50

start_time = time.time()

early_stopping = EarlyStopping(monitor='val_acc', patience=early_stop_after, verbose=1)
checkpointer = ModelCheckpoint(monitor='val_acc', filepath='checkpoints/cnn_images' + '_' + str(size) + '_model.{epoch:05d}-{loss:.4f}-{acc:.4f}-{val_loss:.4f}-{val_acc:.4f}.hdf5', verbose=1, save_best_only=True)

# opt = Adam(lr=learning_rate)
opt = RMSprop(lr=learning_rate, decay=10**-6)
lstm.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
if TRAIN == 1:
    train_history = lstm.fit(X_train, y_train,
                             validation_data=(X_val, y_val),
                             shuffle=True, batch_size=batch_size,
                             epochs=n_epochs,
                             verbose=1,
                             callbacks=[checkpointer, early_stopping])
    
    training_time = time.time() - start_time
    print('> Training time: ' + str(training_time) + ' sec')

# Save model OR Load trained model

In [ ]:
if TRAIN == 1:
    print('> Saving trained model...', end='')
    # Save trained model
    lstm.save_weights(WEIGHT_FILE)
    lstm.save(MODEL_FILE)
    # Save training history
    with open(TRAINING_HIST, 'wb') as f:
        pickle.dump(train_history.history, f)
        
    history_ = train_history.history
    print('done.')
else:
    print('> Loading trained model...', end='')
    # Load trained model
    lstm.load_weights(WEIGHT_FILE)
#     lstm = load_model(MODEL_FILE)
    
    # Load training history
    file = open(TRAINING_HIST, 'rb')
    history_ = pickle.load(file)
    print('done.')

In [ ]:
# Get the latest weight file based on last modified time
lowf = glob.glob('./checkpoints/*.hdf5')
BEST_WEIGHT_FILE = max(lowf, key=os.path.getmtime)
print(BEST_WEIGHT_FILE)

In [ ]:
print('> loading trained model...', end='')
# Load trained model
# lstm = load_model(BEST_WEIGHT_FILE)
lstm.load_weights(BEST_WEIGHT_FILE)

# Load training history
file = open(TRAINING_HIST, 'rb')
history_ = pickle.load(file)
print('done.')

# Validate model again!

In [ ]:
score = lstm.evaluate(X_train, y_train)
print('Train score:', score[0])
print('Train accuracy:', score[1])

score = lstm.evaluate(X_val, y_val)
print('Val score:', score[0])
print('Val accuracy:', score[1])

# Confusion matrix

In [ ]:
y_pred_training_ = lstm.predict(x=X_train, batch_size=batch_size, verbose=1)
y_pred_ = lstm.predict(x=X_val, batch_size=batch_size, verbose=1)

# Saving scores

### Validation

In [ ]:
# Count # of videos
n_vids_val = 0

PATH_VAL_CSV = 'E:/EmotiW2017/lstm/aligned/Val_AFEW_original_scores'

for emo in os.listdir(PATH_VAL):
    full_emo = PATH_VAL + '/' + emo
    if os.path.isdir(full_emo):
        for vid in os.listdir(full_emo):
            n_vids_val += 1

# Read images and extract features
vid_indx = 0

val_names = []

for emo in os.listdir(PATH_VAL):
    full_emo = PATH_VAL + '/' + emo
    if os.path.isdir(full_emo):
        for vid in os.listdir(full_emo):
            full_vid = PATH_VAL_CSV + '/' + emo + '/' + vid
            
            full_vid = full_vid[:-14] + 'csv'

            # Get the name for current video
            val_names.append(full_vid)

            vid_indx += 1

# val_names

In [ ]:
name_indx = 0
cor_pred = 0

for val_f_name in val_names:
    val_scores = y_pred_[name_indx, :]
    cor_pred += (np.argmax(y_pred_[name_indx, :]) == y_val[name_indx])
    
    np.savetxt(val_f_name, val_scores)
    
    name_indx += 1
    
cor_pred/len(y_val)

### Training

In [ ]:
# Count # of videos
n_vids_train = 0

PATH_TRAIN_CSV = 'E:/EmotiW2017/lstm/aligned/Train_AFEW_original_scores'

for emo in os.listdir(PATH_TRAIN):
    full_emo = PATH_TRAIN + '/' + emo
    if os.path.isdir(full_emo):
        for vid in os.listdir(full_emo):
            n_vids_train += 1

# Read images and extract features
vid_indx = 0

train_names = []

for emo in os.listdir(PATH_TRAIN):
    full_emo = PATH_TRAIN + '/' + emo
    if os.path.isdir(full_emo):
        for vid in os.listdir(full_emo):
            full_vid = PATH_TRAIN_CSV + '/' + emo + '/' + vid
            
            full_vid = full_vid[:-14] + 'csv'

            # Get the name for current video
            train_names.append(full_vid)

            vid_indx += 1

# train_names

In [ ]:
name_indx = 0
cor_pred = 0

for train_f_name in train_names:
    train_scores = y_pred_training_[name_indx, :]
    cor_pred += (np.argmax(y_pred_training_[name_indx, :]) == y_train[name_indx])
    
    np.savetxt(train_f_name, train_scores)
    
    name_indx += 1
        
cor_pred/len(y_train)

In [ ]:
y_pred_training = np.argmax(y_pred_training_, axis=1)
y_pred = np.argmax(y_pred_, axis=1)

plot_confusion_matrix(y_test=y_train, y_pred=y_pred_training,
                      classes=classes,
                      normalize=True,
                      title='Normalized confusion matrix - training dataset',
                      cmap=plt.cm.Reds)

plot_confusion_matrix(y_test=y_val, y_pred=y_pred,
                      classes=classes,
                      normalize=True,
                      title='Normalized confusion matrix - validation dataset')

plot_confusion_matrix(y_test=y_val, y_pred=y_pred,
                      classes=classes,
                      normalize=False,
                      title='Unnormalized confusion matrix - validation dataset')

np.sum(y_pred == y_val)/len(y_val)

# Training history

In [ ]:
# list all data in history
print(history_.keys())
# summarize history for accuracy
plt.plot(history_['acc'])
plt.plot(history_['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_['loss'])
plt.plot(history_['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()